In [ ]:
!pip install -q torch torchvision tqdm

In [1]:
# Imports
import math, time, os
from pathlib import Path
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

In [2]:
# Config - tweak for experiments
CFG = {
    'seed': 42,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'batch_size': 128,
    'epochs': 50,           # increase for better accuracy
    'lr': 3e-4,
    'weight_decay': 0.05,
    'patch_size': 4,        # try 4, 8 (4 gives more patches on 32x32)
    'embed_dim': 256,       # model width
    'depth': 12,             # number of transformer blocks
    'num_heads': 8,
    'mlp_ratio': 4.0,       # MLP hidden dim = embed_dim * mlp_ratio
    'dropout': 0.1,
    'num_workers': 2,
    'output_dir': '/content/vit_ckpt'
}
os.makedirs(CFG['output_dir'], exist_ok=True)
torch.manual_seed(CFG['seed'])

In [3]:
# Data transforms (augmentations)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1,0.1,0.1,0.01),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(trainset, batch_size=CFG['batch_size'], shuffle=True, num_workers=CFG['num_workers'], pin_memory=True)
test_loader  = DataLoader(testset, batch_size=CFG['batch_size'], shuffle=False, num_workers=CFG['num_workers'], pin_memory=True)


100%|██████████| 170M/170M [00:03<00:00, 49.5MB/s]


In [4]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_chans=3, embed_dim=256):
        super().__init__()
        assert img_size % patch_size == 0, "img_size must be divisible by patch_size"
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_size = patch_size
        # We implement patchify + linear projection as a conv layer with stride=patch_size
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
    def forward(self, x):
        # x: (B, C, H, W)
        x = self.proj(x)            # (B, embed_dim, H/ps, W/ps)
        x = x.flatten(2).transpose(1,2)  # (B, num_patches, embed_dim)
        return x

In [5]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, dim, num_heads=8, dropout=0.0):
        super().__init__()
        assert dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=True)
        self.proj = nn.Linear(dim, dim)
        self.attn_drop = nn.Dropout(dropout)
        self.proj_drop = nn.Dropout(dropout)

    def forward(self, x):
        # x: (B, N, dim)
        B, N, C = x.shape
        qkv = self.qkv(x)  # (B, N, 3*dim)
        qkv = qkv.reshape(B, N, 3, self.num_heads, self.head_dim).permute(2,0,3,1,4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # each: (B, num_heads, N, head_dim)

        attn = (q @ k.transpose(-2,-1)) * self.scale  # (B, heads, N, N)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = (attn @ v)  # (B, heads, N, head_dim)
        out = out.transpose(1,2).reshape(B,N,C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, dropout=0.0):
        super().__init__()
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, in_features)
        self.drop = nn.Dropout(dropout)
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


In [6]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4.0, dropout=0.1, attn_dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = MultiHeadSelfAttention(dim, num_heads=num_heads, dropout=attn_dropout)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim, int(dim * mlp_ratio), dropout=dropout)
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


In [7]:
class ViT(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_chans=3, num_classes=10,
                 embed_dim=256, depth=8, num_heads=8, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        # cls token
        self.cls_token = nn.Parameter(torch.zeros(1,1,embed_dim))
        # positional embedding
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=dropout)

        # Transformer encoder stack
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_ratio, dropout, attn_dropout=dropout)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)

        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes)

        # init
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)

    def forward(self, x):
        # x: (B, C, H, W)
        B = x.shape[0]
        x = self.patch_embed(x)  # (B, N, dim)
        cls_tokens = self.cls_token.expand(B, -1, -1)  # (B,1,dim)
        x = torch.cat((cls_tokens, x), dim=1)  # (B, N+1, dim)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        cls_out = x[:,0]  # (B, dim)
        logits = self.head(cls_out)
        return logits


In [8]:
device = torch.device(CFG['device'])
model = ViT(img_size=32, patch_size=CFG['patch_size'], embed_dim=CFG['embed_dim'],
            depth=CFG['depth'], num_heads=CFG['num_heads'], mlp_ratio=CFG['mlp_ratio'],
            dropout=CFG['dropout']).to(device)

# Optimizer & scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])

# cosine warmup scheduler
def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, last_epoch=-1):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * num_cycles * 2.0 * progress)))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch)

total_steps = len(train_loader) * CFG['epochs']
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)
criterion = nn.CrossEntropyLoss()


In [9]:
def train_one_epoch(model, loader, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(loader, leave=False)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

        running_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
        pbar.set_description(f"loss: {running_loss/total:.4f} acc: {100*correct/total:.2f}%")
    return running_loss/total, 100*correct/total

def evaluate(model, loader, device):
    model.eval()
    total = 0
    correct = 0
    loss_sum = 0.0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            loss_sum += loss.item() * images.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    return loss_sum/total, 100*correct/total


In [10]:
best_acc = 0.0
for epoch in range(1, CFG['epochs']+1):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, scheduler, device)
    val_loss, val_acc = evaluate(model, test_loader, device)
    t1 = time.time()
    print(f"Epoch {epoch}/{CFG['epochs']}  train_loss={train_loss:.4f} train_acc={train_acc:.2f}%  val_loss={val_loss:.4f} val_acc={val_acc:.2f}%  time={t1-t0:.1f}s")
    # save best
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'model_state': model.state_dict(), 'cfg': CFG, 'epoch': epoch}, os.path.join(CFG['output_dir'], 'best_vit.pth'))
print("Best val acc:", best_acc)


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 1/50  train_loss=1.9752 train_acc=27.07%  val_loss=1.8824 val_acc=31.79%  time=96.3s


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 2/50  train_loss=1.6492 train_acc=38.79%  val_loss=1.4300 val_acc=47.19%  time=96.0s


  0%|          | 0/391 [00:00<?, ?it/s]

...

Epoch 48/50  train_loss=0.3340 train_acc=87.99%  val_loss=0.6044 val_acc=81.34%  time=96.5s


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 49/50  train_loss=0.3270 train_acc=88.25%  val_loss=0.6050 val_acc=81.39%  time=97.0s


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 50/50  train_loss=0.3317 train_acc=87.93%  val_loss=0.6051 val_acc=81.37%  time=96.6s
Best val acc: 81.41
